## COMPUTER VISION PROJECT - Smoke Detector


#### Training YOLO V3
In `Transfer Learning` approach by using pre-trained weights to adopt our task interest saves us from **waiting** for days and huge **computational power**, which woud stand for `cost`.

Transfer learning attempts to teach a neural network by similar means. Rather than training your neural network from scratch, you begin training with preloaded set of weights. Usually you will simply remove the top-most layers of the pretrained neural network and retrain it with new topmost layers. The layers remaining from the previous neural network will be locked so that training does not change these weights. Only the newly added layers will be trained. <br><br>

<html>
<img src="https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/transfer.png" width="75%">
</html>

###Brief Objective:
Create object detection model to detect smoke using train_images with their label txt. file *which are labelled*

### STEPS: 
1. Enable the GPU in the notebook
2. Set up the Cloud: connect Google Drive
3. Clone Darknet and load it
4. Compile the Darknet using GPU
5. Configure the network for training YOLOv3
6. Extracting Training Images
7. Train the detector model

**1. Enable GPU**

Processes dealing with images are mainly computationally expensive; using Cloud computing is one of the efficient ways which is recently booming.

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jul  1 22:33:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**2. Connect to Google Drive**

Cloud environment is set up to yield relatively scalable access to images, weights etc.
Note: We will establish a symbolic connection between  '/content/gdrive/My\ Drive/' ve '/mydrive' because space can cause problem as exists in 'My Drive' or 'Colab Notebooks'.

As drive is mounted, we ignite YOLOv3 to work on the images on Google Drive account by using darknet command. 

In [ ]:
# mount drive by authorization
from google.colab import drive
drive.mount('/content/gdrive')

# this code chunk will create virtual path: /content/gdrive/My \Drive -> /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'			   ML3_Practice1
 Competition_Team			  'NLP Assignment 1'
 Dashboard_Assignment_Arif_Can_Aksoy.zip  'Term3 - Group'
'Getting started.pdf'			   yolov3


**3. Clone the Darknet and Load**

 `Darknet` is an open source neural network framework written in C and CUDA.
 The cells below copies Darknet from AlexeyAB repo. Makefile handles OpenCV and GPU enabling.


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13781, done.
remote: Total 13781 (delta 0), reused 0 (delta 0), pack-reused 13781
Receiving objects: 100% (13781/13781), 12.36 MiB | 5.05 MiB/s, done.
Resolving deltas: 100% (9407/9407), done.


**4. Compile Darknet using GPU**

*stay calm when you see hundreds of warnings as '!make' cell is ran*


In [ ]:
# change makefile to have GPU and OpencCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

In [ ]:
# Verify CUDA
!/usr/local/cuda/bin/nvcc --version

**5. Configure Darknet network for training YOLOv3**

We need to configure Darknet network and make a relation with Google Drive

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

Here we create our layer name. In this case we will create as smoke, given that we want to detect smoke in each image

In [ ]:
!echo "Smoke" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-01 22:39:46--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   470KB/s    in 5m 47s  

2020-07-01 22:45:35 (457 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**6. Extracting Training Images**

The labelled images for training need to be inside a zip archive called "train_images.zip"; and **note** that they need to be inside the folder "yolov3" under root Google Drive

In [ ]:
!unzip /mydrive/yolov3/train_images.zip -d data/obj

Archive:  /mydrive/yolov3/train_images.zip
   creating: data/obj/images/
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3585.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3585.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3586.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3586.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3587.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3587.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3589.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3589.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3590.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3590.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3591.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3591.txt  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3592.jpg  
  inflating: data/obj/images/CameraHPWREN_MirrorSmoke3592.txt  
  inflating: data/obj/images/Ca

Some helper functions

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.43828125 0.34270833333333334 0.8671875 0.5416666666666666

['0', '0.43828125', '0.34270833333333334', '0.8671875', '0.5416666666666666']
0 data/obj/images/MirrorWEBSmoke1852.txt
0 0.43828125 0.34270833333333334 0.8671875 0.5416666666666666
0 0.7328125000000001 0.4513888888888889 0.503125 0.7972222222222223

['0', '0.7328125000000001', '0.4513888888888889', '0.503125', '0.7972222222222223']
1 data/obj/images/CameraHPWREN_MirrorSmoke3904.txt
0 0.7328125000000001 0.4513888888888889 0.503125 0.7972222222222223
0 0.7739583333333333 0.3402777777777778 0.41875 0.6083333333333334

['0', '0.7739583333333333', '0.3402777777777778', '0.41875', '0.6083333333333334']
2 data/obj/images/CameraHPWREN_MirrorSmoke4320.txt
0 0.7739583333333333 0.3402777777777778 0.41875 0.6083333333333334
0 0.8072916666666666 0.4527777777777778 0.36875 0.3666666666666667

['0', '0.8072916666666666', '0.4527777777777778', '0.36875', '0.3666666666666667']
3 data/obj/images/CameraHPWREN_MirrorSmoke4331.txt
0 0.807291666

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/CameraHPWREN_MirrorSmoke3907.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4343.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4301.jpg', 'data/obj/images/MirrorWEBSmoke1538.jpg', 'data/obj/images/MirrorWEBSmoke1866.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4253.jpg', 'data/obj/images/MirrorWEBSmoke1810.jpg', 'data/obj/images/MirrorWEBSmoke1535.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4314.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke3929.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke3913.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4303.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke3905.jpg', 'data/obj/images/MirrorWEBSmoke1828.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke4261.jpg', 'data/obj/images/MirrorWEBSmoke1560.jpg', 'data/obj/images/MirrorWEBSmoke1505.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke3788.jpg', 'data/obj/images/MirrorWEBSmoke1837.jpg', 'data/obj/images/CameraHPWREN_MirrorSmoke3591.jpg', 'data/obj/images/MirrorWEBSmoke1556.jpg

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**7. Start the training**

Finally, we can start training the model. This step should run for 4-6 hours. And complete at least 2500 iterations. In our case we made 3000 (output shows 2480 but it is incomplete), and it took more or less 4 hours

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 252517, rewritten_bbox = 0.000000 % 

 3926: 0.106074, 0.089450 avg loss, 0.001000 rate, 10.414143 seconds, 251264 images, 0.321519 hours left
Loaded: 0.000074 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.831082, GIOU: 0.824096), Class: 0.999710, Obj: 0.937171, No Obj: 0.001756, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.034247, iou_loss = 0.120535, total_loss = 0.154783 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000000, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 tot

 ---End of training---

Now we have our object detection YOLO model. In our Google Drive we have the weights and test file that we will use to predict in new images. For the object detection the notebook called `ObjectDetection` is used.

In [ ]:
!./darknet detector map

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
usage: ./darknet detector [train/test/valid/demo/map] [data] [cfg] [weights (optional)]
